### Fuzzy String Matcher (Basic)

In [12]:
import pandas as pd
import numpy as np
from string_grouper import match_strings, match_most_similar, \
	group_similar_strings, 	StringGrouper
# compute_pairwise_similarities, \
import json
import cleanco

In [13]:
## directory info
rootDataPath = './'

In [33]:
## file read
matchlist = pd.read_csv(f'{rootDataPath}SEC-Data.csv') #from https://www.sec.gov/files/rules/other/4-460list.htm

In [ ]:
def retrieveData():
    url = "https://www.sec.gov/files/company_tickers.json"
    response = requests.get(url)
    json_data = response.json()
    file_name = "company_tickers.json"
    with open(file_name, "w") as json_file:
        json.dump(json_data, json_file)
    print(f"JSON data saved to {file_name}")

def loadData():
    file_name = "company_tickers.json"
    with open(file_name, "r") as json_file:
        loaded_json_data = json.load(json_file)
    df = pd.DataFrame(loaded_json_data)
    df = df.transpose()
    df.title = df.title.str.title()
    df = df.rename({"title":"companyName"},axis=1)
#     df = df[~df.ticker.str.contains('-')]
#     df = df.drop_duplicates(subset='Company', keep='first')
    df.to_csv("SEC-CompanyTicker.csv")
    return df
df = loadData()
df

In [34]:
ref_list=loadData()

## remove the following words from the company names
pattern = '|'.join(['-','_','technologies','technology','pharmaceuticals','partners', 'international',
'financial services','management','consulting','industries','entertainment', 'enterprises','com'
'networks','life sciences','capital markets','communications','resources','healthcare','therapeutics','health care'])

## replace those words
ref_list['dehyphenName'] = ref_list.companyName.str.replace(pattern,' ')

## apply company name cleaning
ref_list['cleanMain'] = ref_list.apply(lambda x: cleanco.basename(x.dehyphenName), axis=1)

/var/folders/nf/2dvvlnxj5v962kwwzmpfj17h0000gn/T/ipykernel_64862/3264793062.py:9: FutureWarning: The default value of regex will change from True to False in a future version.
  ref_list['dehyphenName'] = ref_list.companyName.str.replace(pattern,' ')


In [36]:
## establish list of companies to be matched (the reference list is where we'll be matching them)
to_match_list = matchlist[['companyName']].drop_duplicates().copy().astype(str)
to_match_list['dehyphenName'] = to_match_list.companyName.str.replace(pattern, ' ')
to_match_list['cleanToMatch'] = to_match_list.apply(lambda x: cleanco.basename(x.dehyphenName), axis=1)
to_match_list.head()

/var/folders/nf/2dvvlnxj5v962kwwzmpfj17h0000gn/T/ipykernel_64862/3202591346.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  to_match_list['dehyphenName'] = to_match_list.companyName.str.replace(pattern, ' ')


,companyName,dehyphenName,cleanToMatch
0,3Com Corp,3Com Corp,3Com
1,3M Company,3M Company,3M
2,A.G. Edwards Inc.,A.G. Edwards Inc.,A.G. Edwards
3,Abbott Laboratories,Abbott Laboratories,Abbott Laboratories
4,Abercrombie & Fitch Co.,Abercrombie & Fitch Co.,Abercrombie & Fitch


In [37]:
## break apart the match list to speed things up into blocks of 600k
matchlists_chunks = [to_match_list.cleanToMatch[0:600000].copy(), to_match_list.cleanToMatch[600000:1200000].copy(), to_match_list.cleanToMatch[1200000:].copy()]
matchlists_chunks = [item.astype(str) for item in matchlists_chunks]

In [81]:
# error installing so had to use older version of string_grouper, problem w installing "sparse-dot-topn-for-blocks"

#only returns when finds a particular match so less companies outputted than inputted
## test to check similarity threshold. going with 0.67 for now.
matches1 = match_strings(matchlists_chunks[0],ref_list.cleanMain)
matches1.sort_values(by='similarity', ascending=True).head(5)


,left_side,right_side,similarity
59,Illinois Tool Works,Illinois Tool Works,1.000000
240,Levi Strauss,Levi Strauss,1.000000
128,Martin Marietta Materials,Martin Marietta Materials,1.000000
212,Old Republic International,Old Republic International,1.000000
292,Werner Enterprises,Werner Enterprises,1.000000
...,...,...,...
315,Cars.Com,Harsco,0.809874
331,Th International,Smith International,0.807813
343,Barnes & Noble Education,Barnes & Noble,0.805293
311,Owens & Minor Inc/Va,Owens & Minor,0.802143


In [ ]:
## concatenate all of the matches
matches = pd.concat([match_strings(ref_list.cleanMain, item) for item in matchlists_chunks if len(item)>0], axis=0, ignore_index=True)
matches.sort_values(by='similarity', ascending=False)

In [87]:
#Fuzzy runs slowly by going through each individually
# from fuzzywuzzy import fuzz

# # Define a function to find the most similar match for a company name
# def find_most_similar(name, company_list):
#     similarities = [(existing_name, fuzz.partial_ratio(name, existing_name)) for existing_name in company_list]
#     similarities.sort(key=lambda x: x[1], reverse=True)
#     return similarities[0][0] if similarities and similarities[0][1] >= 10 else None


# # Find matches for each new company in the existing list
# matches = [find_most_similar(company, ref_list.cleanMain) for company in matchlists_chunks[0][:10]]

# # Create a DataFrame to display the results
# results_df = pd.DataFrame({'companyName': matchlists_chunks[0][:10], 'Match': matches})

# # Display the results
# print(results_df)




                companyName                                              Match
0                      3Com                                         Amazon Com
1                        3M                                                 3M
2              A.G. Edwards                                            Wewards
3       Abbott Laboratories                                Abbott Laboratories
4       Abercrombie & Fitch                         Abercrombie & Fitch Co /De
5            ABM Industries                             Abm Industries Inc /De
6              Ace Hardware                                            Radware
7         ACT Manufacturing                                             Manuka
8                   Acterna  Xai Octagon Floating Rate & Alternative Income...
9  Adams Resources & Energy                           Adams Resources & Energy


In [119]:
from sentence_transformers import SentenceTransformer
import faiss
import requests
import json
import pandas as pd
import numpy as np
companyNames = pd.read_csv("SEC-Data.csv")
companyNames = pd.DataFrame({"Company":["Wells Fargo","Sculptor"]}) 


class Similarity:  
    def __init__(self):
        self.possibleCompanies = loadData()

    #train fair with SEC data, SEC data contains duplicate companies, giving more data points for HNSM
    def TrainFaiss(self):
        self.encoder = SentenceTransformer("paraphrase-mpnet-base-v2")
        vectors = self.encoder.encode(list(self.possibleCompanies.companyName))
        vector_dimension = vectors.shape[1]
        self.index = faiss.IndexFlatL2(vector_dimension)
        faiss.normalize_L2(vectors)
        self.index.add(vectors)
    
    #input list of company names and returns dataframe with closest match using FAISS and ticker
    def getCompanyMatch(self, companyNames):
        matches = pd.DataFrame(columns=["Company",'cik_str', 'ticker', 'Match'])
        for i,company in enumerate(companyNames):
            match = self.predict(company)
            matches.loc[i] = [company] + list(match) 
        return matches
            
    def predict(self,search_text):
        search_vector = self.encoder.encode(search_text)
        _vector = np.array([search_vector])
        faiss.normalize_L2(_vector)
        k = self.index.ntotal
        distances, ann = self.index.search(_vector, k=k)
        return self.possibleCompanies.iloc[ann[0][0],:]
    
    

In [120]:

matcher = Similarity()


In [121]:
matcher.TrainFaiss()


In [123]:
matches = matcher.getCompanyMatch(matchlists_chunks[0])

In [124]:
matches

,Company,cik_str,ticker,Match
0,3Com,66740,MMM,3M Co
1,3M,66740,MMM,3M Co
2,A.G. Edwards,1099800,EW,Edwards Lifesciences Corp
3,Abbott Laboratories,1800,ABT,Abbott Laboratories
4,Abercrombie & Fitch,1018840,ANF,Abercrombie & Fitch Co /De/
...,...,...,...,...
942,Yellow,716006,YELLQ,Yellow Corp
943,York International,108985,YORW,York Water Co
944,Yum Brands,1041061,YUM,Yum Brands Inc
945,Zale,1893124,ZAAG,"Za Group, Inc."


In [128]:
matches.head(30)

,Company,cik_str,ticker,Match
0,3Com,66740,MMM,3M Co
1,3M,66740,MMM,3M Co
2,A.G. Edwards,1099800,EW,Edwards Lifesciences Corp
3,Abbott Laboratories,1800,ABT,Abbott Laboratories
4,Abercrombie & Fitch,1018840,ANF,Abercrombie & Fitch Co /De/
5,ABM Industries,771497,ABM,Abm Industries Inc /De/
6,Ace Hardware,1841925,INDI,"Indie Semiconductor, Inc."
7,ACT Manufacturing,7623,ARTW,Arts Way Manufacturing Co Inc
8,Acterna,1388320,ATNM,"Actinium Pharmaceuticals, Inc."
9,Adams Resources & Energy,2178,AE,"Adams Resources & Energy, Inc."


In [127]:
possibleCompanies = pd.read_csv("SEC-CompanyTicker.csv",index_col=0)
matcher.getCompanyMatch(possibleCompanies.companyName[:10])

,Company,cik_str,ticker,Match
0,Apple Inc.,320193,AAPL,Apple Inc.
1,Microsoft Corp,789019,MSFT,Microsoft Corp
2,Alphabet Inc.,1652044,GOOGL,Alphabet Inc.
3,Amazon Com Inc,1018724,AMZN,Amazon Com Inc
4,Nvidia Corp,1045810,NVDA,Nvidia Corp
5,"Tesla, Inc.",1318605,TSLA,"Tesla, Inc."
6,Berkshire Hathaway Inc,1067983,BRK-B,Berkshire Hathaway Inc
7,"Meta Platforms, Inc.",1326801,META,"Meta Platforms, Inc."
8,Eli Lilly & Co,59478,LLY,Eli Lilly & Co
9,Visa Inc.,1403161,V,Visa Inc.
